In [4]:
!pip install -q torch==1.10.1 torchvision 

In [26]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.nn.parameter import Parameter
import torchvision
import numpy as np
from SpykeTorch import snn
from SpykeTorch import functional as sf
from SpykeTorch import visualization as vis
from SpykeTorch import utils
from torchvision import transforms

#use_cuda = True
use_cuda = False

In [27]:
class fly_eye(nn.Module):
    def __init__(self):
        super(fly_eye, self).__init__()

        # photoreceptor cells
        self.R1_6 = snn.Convolutional(1,1,1)
        self.R7 = snn.Convolutional(1,1,1)
        self.R8 = snn.Convolutional(1,1,1)

        # Lamina
        self.L1 = snn.Convolutional(1,1,1)
        self.L2 = snn.Convolutional(1,1,1)
        self.L3 = snn.Convolutional(1,1,1)
        self.L4 = snn.Convolutional(1,1,1)
        self.L5 = snn.Convolutional(1,1,1)
        
        self.C2 = snn.Convolutional(1,1,1)
        self.C3 = snn.Convolutional(1,1,1)
        
        # Medulla
        self.Mi1 = snn.Convolutional(1,1,1)
        self.Mi4 = snn.Convolutional(1,1,1)
        self.Mi9 = snn.Convolutional(1,1,1)
        self.Mi15 = snn.Convolutional(1,1,1)
        
        self.Tm1 = snn.Convolutional(1,1,1)
        self.Tm2 = snn.Convolutional(1,1,1)
        self.Tm3 = snn.Convolutional(1,1,1)
        self.Tm4 = snn.Convolutional(1,1,1)
        self.Tm6 = snn.Convolutional(1,1,1)
        self.Tm9 = snn.Convolutional(1,1,1)
        self.Tm20 = snn.Convolutional(1,1,1)
        self.TmY5a = snn.Convolutional(1,1,1)
        
        self.T2 = snn.Convolutional(1,1,1)
        self.T2a = snn.Convolutional(1,1,1)
        self.T3 = snn.Convolutional(1,1,1)
        
        # Lobula
        self.LC4 = snn.Convolutional(1,1,1)
        self.LC17 = snn.Convolutional(1,1,1)
        
        # Cenral brain
        self.central_brain = snn.Convolutional(1,1,1)
        
        
        # STDP applying
        self.stdpR1_6 = snn.STDP(self.R1_6, (0.004, -0.003))
        self.stdpR7 = snn.STDP(self.R7, (0.004, -0.003))
        self.stdpR8 = snn.STDP(self.R8, (0.004, -0.003))        
        
        self.stdpL1 = snn.STDP(self.L1, (0.004, -0.003))
        self.stdpL2 = snn.STDP(self.L2, (0.004, -0.003))
        self.stdpL3 = snn.STDP(self.L3, (0.004, -0.003))
        self.stdpL4 = snn.STDP(self.L4, (0.004, -0.003))
        self.stdpL5 = snn.STDP(self.L5, (0.004, -0.003))
        
        self.stdpC2 = snn.STDP(self.C2, (0.004, -0.003))
        self.stdpC3 = snn.STDP(self.C3, (0.004, -0.003))
        
        self.stdpMi1 = snn.STDP(self.Mi1, (0.004, -0.003))
        self.stdpMi14 = snn.STDP(self.Mi4, (0.004, -0.003))
        self.stdpMi9 = snn.STDP(self.Mi9, (0.004, -0.003))
        self.stdpMi15 = snn.STDP(self.Mi15, (0.004, -0.003))
        
        self.stdpTm1 = snn.STDP(self.Tm1, (0.004, -0.003))
        self.stdpTm2 = snn.STDP(self.Tm2, (0.004, -0.003))
        self.stdpTm3 = snn.STDP(self.Tm3, (0.004, -0.003))
        self.stdpTm4 = snn.STDP(self.Tm4, (0.004, -0.003))
        self.stdpTm6 = snn.STDP(self.Tm6, (0.004, -0.003))
        self.stdpTm9 = snn.STDP(self.Tm9, (0.004, -0.003))
        self.stdpTm20 = snn.STDP(self.Tm20, (0.004, -0.003))
        self.stdpTmY5a = snn.STDP(self.TmY5a, (0.004, -0.003))
        
        self.stdpT2 = snn.STDP(self.T2, (0.004, -0.003))
        self.stdpT2a = snn.STDP(self.T2a, (0.004, -0.003))
        self.stdpT3 = snn.STDP(self.T3, (0.004, -0.003))
        
        self.stdpLC4 = snn.STDP(self.LC4, (0.004, -0.003))
        self.stdpLC17 = snn.STDP(self.LC17, (0.004, -0.003))
        
        self.anti_stdp_central_brain = snn.STDP(self.central_brain, (-0.004, 0.005), False, 0.2, 0.8)
        self.stdp_central_brain = snn.STDP(self.central_brain, (0.004, -0.003), False, 0.2, 0.8)
        
        
        
        
        self.ctx = {"input_spikes":None, "potentials":None, "output_spikes":None, "winners":None}
        self.spk_cnt1 = 0
        self.spk_cnt2 = 0

        
        
    def forward(self, input, max_layer):
        # input - данные после обработки
        
        # R1_6, R7, R8-------------------------------------------------------------------------------------------------
        R1_6 = input

        potR7 = self.convR7(input)
        spkR7, potR7 = sf.fire(potR7, self.convR7_t, True)

        potR8 = self.convR8(input)
        spkR8, potR8 = sf.fire(potR8, self.convR8_t, True)

        # L1, L2, L3, L4, L5------------------------------------------------------------------------------------------
        # L1: R1_6 + R8 as input
        potL1 = self.convL1(torch.cat([R1_6,spkR8], dim=1))
        spkL1, potL1 = sf.fire(potL1, self.convL1_t, True)
        # L2: R1_6 as input
        potL2 = self.convL2(R1_6)
        spkL2, potL2 = sf.fire(potL2, self.convL2_t, True)
        # L3: R1_6, R7, R8 as input
        potL3 = self.convL3(torch.cat([R1_6,spkR7,spkR8], dim=1))
        spkL3, potL3 = sf.fire(potL3, self.convL3_t, True)
        # L4: R1_6 as input
        potL4 = self.convL4(R1_6)
        spkL4, potL4 = sf.fire(potL4, self.convL4_t, True)
        # L5: R1_6 + R8 as input
        potL5 = self.convL5(torch.cat([R1_6,spkR8], dim=1))
        spkL5, potL5 = sf.fire(potL5, self.convL5_t, True)

        # C2, C3------------------------------------------------------------------------------------------------------
        # C2: L1, L5(5,5) as input
        potC2 = self.convC2(torch.cat([spkL1,
                                       sf.pooling(spkL5, 5, 1, 2)],
                                      dim=1))
        spkC2, potC2 = sf.fire(potC2, self.convC2_t, True)
        # C3: L1, L2, L3, L5(3,3) as input
        potC3 = self.convC3(torch.cat([spkL1, spkL2, spkL3,
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkC3, potC3 = sf.fire(potC3, self.convC3_t, True)

        # Mi1, Mi4, Mi9, Mi15-----------------------------------------------------------------------------------------
        # Mi1: R8, L1, L3, C2, C3, L5(3,3) as input
        potMi1 = self.convMi1(torch.cat([spkR8, spkL1, spkL3,spkC2,spkC3,
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkMi1, potMi1 = sf.fire(potMi1, self.convMi1_t, True)
        # Mi4: R8, L2, L3, C2, C3, L5(3,3) as input
        potMi4 = self.convMi4(torch.cat([spkR8, spkL2, spkL3,spkC2,spkC3, 
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkMi4, potMi4 = sf.fire(potMi4, self.convMi4_t, True)
        # Mi9: R7, R8, L2, L3(3,3), L4(3,3) as input
        potMi9 = self.convMi9(torch.cat([spkR7, spkR8, spkL2, 
                                       sf.pooling(spkL3, 3, 1, 1),
                                       sf.pooling(spkL4, 3, 1, 1)],
                                      dim=1))
        spkMi9, potMi9 = sf.fire(potMi9, self.convMi9_t, True)
        # Mi15: R8(3,3), L5(3,3) as input
        potMi15 = self.convMi15(torch.cat([ 
                                       sf.pooling(spkR8, 3, 1, 1),
                                       sf.pooling(spkL5, 3, 1, 1)],
                                      dim=1))
        spkMi15, potMi15 = sf.fire(potMi15, self.convMi15_t, True)

        # Tm1, Tm2, Tm3, Tm4, Tm6, Tm9, Tm20, TmY5a------------------------------------------------------------------
        # Tm1: L2, L5, C2, C3, Mi1, Mi4, Mi9
        potTm1 = self.convTm1(torch.cat([spkL2, spkL5, spkC2, spkC3,
                                         spkMi1, spkMi4, spkMi9]
                                      dim=1))
        spkTm1, potTm1 = sf.fire(potTm1, self.convTm1_t, True)
        # Tm2: L2, L4(3,3), C3, Mi1, Mi9
        potTm2 = self.convTm2(torch.cat([spkL2, spkC3, spkMi1, spkMi9,
                                         sf.pooling(spkL4, 3, 1, 1)]
                                      dim=1))
        spkTm2, potTm2 = sf.fire(potTm2, self.convTm2_t, True)
        # Tm3: L3(3,3), C2(3,3), Mi4(3,3), Mi9(3,3), L1(5,5), L5(5,5), Mi1(5,5)
        potTm3 = self.convTm3(torch.cat([
                                         sf.pooling(spkL3, 3, 1, 1),
                                         sf.pooling(spkC2, 3, 1, 1),
                                         sf.pooling(spkMi4, 3, 1, 1),
                                         sf.pooling(spkMi9, 3, 1, 1),                
                                         sf.pooling(spkL1, 5, 1, 2),
                                         sf.pooling(spkL5, 5, 1, 2),
                                         sf.pooling(spkMi1, 5, 1, 2)]
                                      dim=1))
        spkTm3, potTm3 = sf.fire(potTm3, self.convTm3_t, True)
        # Tm4: L4(3,3), Mi4(3), Mi9(3), L2(5), C3(5)
        potTm4 = self.convTm4(torch.cat([
                                         sf.pooling(spkL4, 3, 1, 1),
                                         sf.pooling(spkMi4, 3, 1, 1),
                                         sf.pooling(spkMi9, 3, 1, 1),                
                                         sf.pooling(spkL2, 5, 1, 2),
                                         sf.pooling(spkC3, 5, 1, 2)]
                                      dim=1))
        spkTm4, potTm4 = sf.fire(potTm4, self.convTm4_t, True)
        # Tm6: Mi1(3), Mi15(3), L5(5), Mi9(5)
        potTm6 = self.convTm6(torch.cat([sf.pooling(spkMi1, 3, 1, 1),
                                         sf.pooling(spkMi15, 3, 1, 1),               
                                         sf.pooling(spkL5, 5, 1, 2),
                                         sf.pooling(spkMi9, 5, 1, 2)]
                                      dim=1))
        spkTm6, potTm6 = sf.fire(potTm6, self.convTm6_t, True)
        # Tm9: L4(3), Mi4(3), L2, C2, C3
        potTm9 = self.convTm9(torch.cat([sf.pooling(spkL4, 3, 1, 1),
                                        sf.pooling(spkMi4, 3, 1, 1),
                                        spkL2, spkC2, spkC3]
                                      dim=1))
        spkTm9, potTm9 = sf.fire(potTm9, self.convTm9_t, True)
        # Tm20: Mi4(3), R7, R8, L2, C3, Mi1
        potTm20 = self.convTm20(torch.cat([sf.pooling(spkMi4, 3, 1, 1),
                                        spkR7, spkR8, spkL2, spkC3, spkMi1]
                                      dim=1))
        spkTm20, potTm20 = sf.fire(potTm20, self.convTm20_t, True)
        # TmY5a: L5(3), Mi4(3), Mi9(3)
        potTmY5a = self.convTmY5a(torch.cat([sf.pooling(spkL5, 3, 1, 1),
                                            sf.pooling(spkMi4, 3, 1, 1),
                                            sf.pooling(spkMi9, 3, 1, 1)]
                                      dim=1))
        spkTmY5a, potTmY5a = sf.fire(potTmY5a, self.convTmY5a_t, True)

        # T2, T2a, T3-----------------------------------------------------------------------------------------------
        # T2: Mi1(3), Tm1(3), Tm3(3), Tm4(3), TmY5a(3)
        potT2 = self.convT2(torch.cat([sf.pooling(spkMi1, 3, 1, 1),
                                            sf.pooling(spkTm1, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm4, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1)]
                                      dim=1))
        spkT2, potT2 = sf.fire(potT2, self.convT2_t, True)
        # T2a: L2, L5, C2, C3, Mi4, Tm1, Tm2
        potT2a = self.convT2a(torch.cat([spkL2, spkL5, spkC2, spkC3, spkMi4, spkTm1, spkTm2], dim=1))
        spkT2a, potT2a = sf.fire(potT2a, self.convT2a_t, True)
        # T3: L2, L4, L5, C2, C3, Mi1, Mi9, Tm1, Tm2, Tm3, Tm6, TmY5a - 3 window
        potT3 = self.convT3(torch.cat([sf.pooling(spkL2, 3, 1, 1),
                                            sf.pooling(spkL4, 3, 1, 1),
                                            sf.pooling(spkL5, 3, 1, 1),
                                            sf.pooling(spkC2, 3, 1, 1),
                                            sf.pooling(spkC3, 3, 1, 1),
                                            sf.pooling(spkMi1, 3, 1, 1),
                                            sf.pooling(spkMi9, 3, 1, 1),
                                            sf.pooling(spkTm1, 3, 1, 1),
                                            sf.pooling(spkTm2, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm6, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1)]
                                      dim=1))
        spkT3, potT3 = sf.fire(potT3, self.convT3_t, True)

        # LC4, LC17------------------------------------------------------------------------------------------------
        # LC4: Tm1, Tm2, Tm3, Tm4, Tm6, Tm9, TmY5a, T2, T2a, T3 - 5 window
        potLC4 = self.convLC4(torch.cat([sf.pooling(spkTm1, 5, 1, 2),
                                            sf.pooling(spkTm2, 5, 1, 2),
                                            sf.pooling(spkTm3, 5, 1, 2),
                                            sf.pooling(spkTm4, 5, 1, 2),
                                            sf.pooling(spkTm6, 5, 1, 2),
                                            sf.pooling(spkTm9, 5, 1, 2),
                                            sf.pooling(spkTmY5a, 5, 1, 2),
                                            sf.pooling(spkT2, 5, 1, 2),
                                            sf.pooling(spkT2a, 5, 1, 2),
                                            sf.pooling(spkT3, 5, 1, 2)]
                                      dim=1))
        spkLC4, potLC4 = sf.fire(potLC4, self.convLC4_t, True)
        # LC17: Tm2, Tm3, Tm4, Tm6, Tm9, Tm20, TmY5a, T2, T2a, T3 - 3 window
        potLC17 = self.convLC17(torch.cat([sf.pooling(spkTm2, 3, 1, 1),
                                            sf.pooling(spkTm3, 3, 1, 1),
                                            sf.pooling(spkTm4, 3, 1, 1),
                                            sf.pooling(spkTm6, 3, 1, 1),
                                            sf.pooling(spkTm9, 3, 1, 1),
                                            sf.pooling(spkTm20, 3, 1, 1),
                                            sf.pooling(spkTmY5a, 3, 1, 1),
                                            sf.pooling(spkT2, 3, 1, 1),
                                            sf.pooling(spkT2a, 3, 1, 1),
                                            sf.pooling(spkT3, 3, 1, 1)]
                                      dim=1))
        spkLC17, potLC17 = sf.fire(potLC17, self.convLC17_t, True)

        # central brain---------------------------------------------------------------------------------------------
        potCB = self.convCB(torch.cat([spkLC4, spkLC17]
                                      dim=1))
        spkCB, potCB = sf.fire(potCB)

        if self.training:    
            # finish training
            winners = sf.get_k_winners(potCB, 1, 0, spkCB)
            self.ctx["input_spikes"] = torch.cat([spkLC4, spkLC17]
                                          dim=1)
            self.ctx["potentials"] = potCB
            self.ctx["output_spikes"] = spkCB
            self.ctx["winners"] = winners
            output = -1
            if len(winners) != 0:
                output = self.decision_map[winners[0][0]]
            return outputs
        else:
            winners = sf.get_k_winners(potCB, 1, 0, spkCB)
            output = -1
            if len(winners) != 0:
                output = self.decision_map[winners[0][0]]
            return output
        
        
        
        
        
        
        
        
        
        
    
    def stdp(self, layer_idx):
        if layer_idx == 1:
            self.stdp1(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])
        if layer_idx == 2:
            self.stdp2(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])

    def update_learning_rates(self, stdp_ap, stdp_an, anti_stdp_ap, anti_stdp_an):
        self.stdp3.update_all_learning_rate(stdp_ap, stdp_an)
        self.anti_stdp3.update_all_learning_rate(anti_stdp_an, anti_stdp_ap)

    def reward(self):
        self.stdp3(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])

    def punish(self):
        self.anti_stdp3(self.ctx["input_spikes"], self.ctx["potentials"], self.ctx["output_spikes"], self.ctx["winners"])

In [28]:
def train_unsupervise(network, data, layer_idx):
    network.train()
    for i in range(len(data)):
        data_in = data[i]
        if use_cuda:
            data_in = data_in.cuda()
        network(data_in, layer_idx)
        network.stdp(layer_idx)

In [29]:
def train_rl(network, data, target):
    network.train()
    perf = np.array([0,0,0]) # correct, wrong, silence
    for i in range(len(data)):
        data_in = data[i]
        target_in = target[i]
        if use_cuda:
            data_in = data_in.cuda()
            target_in = target_in.cuda()
        d = network(data_in, 3)
        if d != -1:
            if d == target_in:
                perf[0]+=1
                network.reward()
            else:
                perf[1]+=1
                network.punish()
        else:
            perf[2]+=1
    return perf/len(data)

In [30]:
def test(network, data, target):
    network.eval()
    perf = np.array([0,0,0]) # correct, wrong, silence
    for i in range(len(data)):
        data_in = data[i]
        target_in = target[i]
        if use_cuda:
            data_in = data_in.cuda()
            target_in = target_in.cuda()
        d = network(data_in, 3)
        if d != -1:
            if d == target_in:
                perf[0]+=1
            else:
                perf[1]+=1
        else:
            perf[2]+=1
    return perf/len(data)

In [31]:
class S1C1Transform:
    def __init__(self, filter, timesteps = 15):
        self.to_tensor = transforms.ToTensor()
        self.filter = filter
        self.temporal_transform = utils.Intensity2Latency(timesteps)
        self.cnt = 0
    def __call__(self, image):
        if self.cnt % 1000 == 0:
            print(self.cnt)
        self.cnt+=1
        image = self.to_tensor(image) * 255
        image.unsqueeze_(0)
        image = self.filter(image)
        image = sf.local_normalization(image, 8)
        temporal_image = self.temporal_transform(image)
        return temporal_image.sign().byte()


In [ ]:
kernels = [ utils.DoGKernel(3,3/9,6/9),
            utils.DoGKernel(3,6/9,3/9),
            utils.DoGKernel(7,7/9,14/9),
            utils.DoGKernel(7,14/9,7/9),
            utils.DoGKernel(13,13/9,26/9),
            utils.DoGKernel(13,26/9,13/9)]
filter = utils.Filter(kernels, padding = 6, thresholds = 50)
s1c1 = S1C1Transform(filter)

data_root = "data"
MNIST_train = utils.CacheDataset(torchvision.datasets.MNIST(root=data_root, train=True, download=True, transform = s1c1))
MNIST_test = utils.CacheDataset(torchvision.datasets.MNIST(root=data_root, train=False, download=True, transform = s1c1))
MNIST_loader = DataLoader(MNIST_train, batch_size=1000, shuffle=False)
MNIST_testLoader = DataLoader(MNIST_test, batch_size=len(MNIST_test), shuffle=False)

mozafari = MozafariMNIST2018()
if use_cuda:
    mozafari.cuda()

# Training The First Layer
print("Training the first layer")
if os.path.isfile("saved_l1.net"):
    mozafari.load_state_dict(torch.load("saved_l1.net"))
else:
    for epoch in range(2):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(mozafari, data, 1)
            print("Done!")
            iter+=1
    torch.save(mozafari.state_dict(), "saved_l1.net")
# Training The Second Layer
print("Training the second layer")
if os.path.isfile("saved_l2.net"):
    mozafari.load_state_dict(torch.load("saved_l2.net"))
else:
    for epoch in range(4):
        print("Epoch", epoch)
        iter = 0
        for data,targets in MNIST_loader:
            print("Iteration", iter)
            train_unsupervise(mozafari, data, 2)
            print("Done!")
            iter+=1
    torch.save(mozafari.state_dict(), "saved_l2.net")

# initial adaptive learning rates
apr = mozafari.stdp3.learning_rate[0][0].item()
anr = mozafari.stdp3.learning_rate[0][1].item()
app = mozafari.anti_stdp3.learning_rate[0][1].item()
anp = mozafari.anti_stdp3.learning_rate[0][0].item()

adaptive_min = 0
adaptive_int = 1
apr_adapt = ((1.0 - 1.0 / 10) * adaptive_int + adaptive_min) * apr
anr_adapt = ((1.0 - 1.0 / 10) * adaptive_int + adaptive_min) * anr
app_adapt = ((1.0 / 10) * adaptive_int + adaptive_min) * app
anp_adapt = ((1.0 / 10) * adaptive_int + adaptive_min) * anp

# perf
best_train = np.array([0.0,0.0,0.0,0.0]) # correct, wrong, silence, epoch
best_test = np.array([0.0,0.0,0.0,0.0]) # correct, wrong, silence, epoch

# Training The Third Layer
print("Training the third layer")
for epoch in range(680):
    print("Epoch #:", epoch)
    perf_train = np.array([0.0,0.0,0.0])
    for data,targets in MNIST_loader:
        perf_train_batch = train_rl(mozafari, data, targets)
        print(perf_train_batch)
        #update adaptive learning rates
        apr_adapt = apr * (perf_train_batch[1] * adaptive_int + adaptive_min)
        anr_adapt = anr * (perf_train_batch[1] * adaptive_int + adaptive_min)
        app_adapt = app * (perf_train_batch[0] * adaptive_int + adaptive_min)
        anp_adapt = anp * (perf_train_batch[0] * adaptive_int + adaptive_min)
        mozafari.update_learning_rates(apr_adapt, anr_adapt, app_adapt, anp_adapt)
        perf_train += perf_train_batch
    perf_train /= len(MNIST_loader)
    if best_train[0] <= perf_train[0]:
        best_train = np.append(perf_train, epoch)
    print("Current Train:", perf_train)
    print("   Best Train:", best_train)

    for data,targets in MNIST_testLoader:
        perf_test = test(mozafari, data, targets)
        if best_test[0] <= perf_test[0]:
            best_test = np.append(perf_test, epoch)
            torch.save(mozafari.state_dict(), "saved.net")
        print(" Current Test:", perf_test)
        print("    Best Test:", best_test)

Training the first layer
Epoch 0
0
Iteration 0
Done!
1000
Iteration 1
Done!
2000
Iteration 2
Done!
3000
Iteration 3
Done!
4000
Iteration 4
Done!
5000
Iteration 5
Done!
6000
Iteration 6
Done!
7000
Iteration 7
Done!
8000
Iteration 8
Done!
9000
Iteration 9
Done!
10000
Iteration 10
Done!
11000
Iteration 11
Done!
12000
Iteration 12
Done!
13000
Iteration 13
Done!
14000
Iteration 14
Done!
15000
Iteration 15
Done!
16000
Iteration 16
Done!
17000
Iteration 17
Done!
18000
Iteration 18
Done!
19000
Iteration 19
Done!
20000
Iteration 20
Done!
21000
Iteration 21
Done!
22000
Iteration 22
Done!
23000
Iteration 23
Done!
24000
Iteration 24
Done!
25000
Iteration 25
Done!
26000
Iteration 26
Done!
27000
Iteration 27
Done!
28000
Iteration 28
Done!
29000
Iteration 29
Done!
30000
Iteration 30
Done!
31000
Iteration 31
Done!
32000
Iteration 32
Done!
33000
Iteration 33
Done!
34000
Iteration 34
Done!
35000
Iteration 35
Done!
36000
Iteration 36
Done!
37000
Iteration 37
Done!
38000
Iteration 38
Done!
39000
Iteration